In [ ]:
## Matching Notebook

## Match MHSAA baseball record data with school demo data

In [1]:
import pandas as pd
from fuzzywuzzy import process

## Load baseball game data


game_df = pd.read_csv('MHSAA_team_stats_Thu_Jul_22_2021_11-30.csv')


In [ ]:
# game_df.head()
# df.info()


In [2]:
## List primary team names

game_teams = game_df['primary_team'].unique().tolist()

len(game_teams)



602

In [3]:
## open the school demographic data from the finished df

school_info_df = pd.read_csv('school_info_df.csv')



# df_public = pd.read_csv('data\MHSAA\school_info\school_info_PUBLIC.csv')
# df_private = pd.read_csv('data\MHSAA\school_info\school_info_PRIVATE.csv')

In [4]:
# df_public.columns

Index(['School Name', 'NCES School ID', 'County Name', 'Street Address',
       'City', 'State', 'ZIP', 'ZIP 4-digit', 'Phone', 'Locale Code', 'Locale',
       'Charter', 'Magnet', 'Title I School', 'Title 1 School Wide',
       'Students', 'Teachers', 'Student Teacher Ratio', 'Free Lunch',
       'Reduced Lunch', 'Directly Certified'],
      dtype='object')

In [5]:
# ## Add a column to each dataframe that indicates the type of school
# df_public['school_type'] = 'public'
# df_private['school_type'] = 'private'

In [ ]:
# # df_public.head()
# df_public.info()



# len(school_names)



In [6]:
# df_private.head()
# df_private.info()

# # Create a list of  public names
# school_names = df_public['School Name'].unique().tolist()
# # Create a list of private names
# private_school_names = df_private['PSS_INST'].unique().tolist()

# # add the private school names to the public school names
# school_names = school_names + private_school_names

# ## transform the school names to lower case
# school_names = [x.lower() for x in school_names]

# len(school_names)

# # len(private_school_names)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 60 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   PSS_SCHOOL_ID    156 non-null    object 
 1   PSS_INST         156 non-null    object 
 2   LoGrade          156 non-null    int64  
 3   HiGrade          156 non-null    int64  
 4   PSS_ADDRESS      156 non-null    object 
 5   PSS_CITY         156 non-null    object 
 6   PSS_COUNTY_NO    156 non-null    int64  
 7   PSS_COUNTY_FIPS  156 non-null    int64  
 8   PSS_STABB        156 non-null    object 
 9   PSS_FIPS         156 non-null    int64  
 10  PSS_ZIP5         156 non-null    int64  
 11  PSS_PHONE        155 non-null    float64
 12  PSS_SCH_DAYS     153 non-null    float64
 13  PSS_STU_DAY_HRS  156 non-null    float64
 14  PSS_LIBRARY      156 non-null    object 
 15  PSS_ENROLL_UG    6 non-null      float64
 16  PSS_ENROLL_PK    72 non-null     float64
 17  PSS_ENROLL_K    

1451

In [7]:
# Transform the Private school data to more readable format

# Create a dictionary mapping locale codes to descriptions
locale_dict = {
    11: 'City: Large',
    12: 'City: Midsize',
    13: 'City: Small',
    21: 'Suburb: Large',
    22: 'Suburb: Midsize',
    23: 'Suburb: Small',
    31: 'Town: Fringe',
    32: 'Town: Distant',
    33: 'Town: Remote',
    41: 'Rural: Fringe',
    42: 'Rural: Distant',
    43: 'Rural: Remote'
}

relig_dict = {
    1: 'Catholic',
    2: 'Other Religious',
    3: 'Nonsectarian'

}

ort_dict = {
    1: 'Roman Catholic',
    2: 'African Methodist Episcopal',
    3: 'Amish',
    4: 'Assembly of God',
    5: 'Baptist',
    6: 'Brethren',
    7: 'Calvinist',
    8: 'Christian (no denomination specified)',
    9: 'Church of Christ',
    10: 'Church of God',
    11: 'Church of God in Christ',
    12: 'Church of the Nazarene',
    13: 'Disciples of Christ',
    14: 'Episcopal',
    15: 'Friends',
    16: 'Greek Orthodox',
    17: 'Islamic',
    18: 'Jewish',
    19: 'Latter Day Saints',
    20: 'Lutheran Church – Missouri Synod',
    21: 'Evangelical Lutheran Church in America',
    22: 'Wisconsin Evangelical Lutheran Synod',
    23: 'Other Lutheran',
    24: 'Mennonite',
    25: 'Methodist',
    26: 'Penecostal',
    27: 'Presbyterian',
    28: 'Seventh Day Adventists',
    29: 'Other',
    30: 'Nonsectarians'
}

## Change PSS_COMM_TYPE to a more readable format
# This is an higher level classification of Locale

# Create a dictionary mapping locale codes to descriptions
comm_dict = {
    1: 'City',
    2: 'Suburban',
    3: 'Town',
    4: 'Rural'
}

# Dictionary for COOED values
coed_dict = {
    1: 'Coed',
    2: 'Female_Only',
    3: 'Male_Only'
}


# Apply the mapping to the columns
df_private['PSS_COED'] = df_private['PSS_COED'].map(coed_dict)
df_private['PSS_LOCALE'] = df_private['PSS_LOCALE'].map(locale_dict)
df_private['PSS_COMM_TYPE'] = df_private['PSS_COMM_TYPE'].map(comm_dict)
df_private['PSS_ORIENT'] = df_private['PSS_ORIENT'].map(ort_dict)
df_private['PSS_RELIG'] = df_private['PSS_RELIG'].map(relig_dict)


# Calculate the High School Enrollment (Grades 9-12)
cols_to_sum = ['PSS_ENROLL_9', 'PSS_ENROLL_10', 'PSS_ENROLL_11', 'PSS_ENROLL_12', 'PSS_ENROLL_UG']
df_private['HS_ENROLL'] = df_private[cols_to_sum].fillna(0).sum(axis=1)





In [8]:
## Get the list of teams from the game data and create a list of unique teams

game_teams = game_df['primary_team'].unique().tolist()

# strip and LC for matching
game_teams = [x.lower() for x in game_teams]

len(game_teams)

602

In [ ]:
# Renaming columns in df_private to match df_public
df_private = df_private.rename(columns={
    'PSS_INST': 'School Name',
    'PSS_ADDRESS': 'Street Address',
    'PSS_CITY': 'City',
    'PSS_PHONE': 'Phone',
    'PSS_ZIP5': 'ZIP',
    'PSS_ZIP4': 'ZIP 4-digit',
    'PSS_COUNTY_NAME': 'County Name',
    'PSS_COUNTY_FIPS': 'County FIPS',
    'PSSCOUNTY_NO': 'County Number',
    'PSS_STABB': 'State',
    'PSS_ORIENT': 'Relig_subtype',
    'PSS_RELIG': 'Relig',
    'PSS_LOCALE': 'Locale',
    'PSS_COMM_TYPE': 'Community_Type',
    'PSS_INDIAN_PCT': 'demo_pct_American Indian',
    'PSS_ASIAN_PCT': 'demo_pct_Asian',
    'PSS_HISP_PCT': 'demo_pct_Hispanic',
    'PSS_BLACK_PCT': 'demo_pct_Black',
    'PSS_WHITE_PCT': 'demo_pct_White',
    'PSS_PACISL_PCT': 'demo_pct_Pacific Islander',
    'PSS_TWOMORE_PCT': 'demo_pct_Two_or_More',

    'PSS_COED': 'Coed?',

    'PSS_STDTCH_RT': 'Student Teacher Ratio',
    'PSS_SCHOOL_ID' : 'School ID(private)',
   
})

# Drop the columns that are not needed
drop_col = ['LoGrade','HiGrade','PSS_RACE_AI','PSS_RACE_AS','PSS_RACE_H',
            'PSS_RACE_B','PSS_RACE_W','PSS_RACE_P','PSS_RACE_2','PSS_TYPE',
            'PSS_LEVEL','PSS_ASSOC_2','PSS_ASSOC_3','PSS_FIPS','PSS_SCH_DAYS',
            'PSS_STU_DAY_HRS','PSS_LIBRARY','PSS_ENROLL_UG','PSS_ENROLL_PK',
            'PSS_ENROLL_K','PSS_ENROLL_1','PSS_ENROLL_2','PSS_ENROLL_3','PSS_ENROLL_4',
            'PSS_ENROLL_5','PSS_ENROLL_6','PSS_ENROLL_7','PSS_ENROLL_8','PSS_ENROLL_9',
            'PSS_ENROLL_10','PSS_ENROLL_11','PSS_ENROLL_12','PSS_ENROLL_T','PSS_ENROLL_TK12',
            'PSS_FTE_TEACH'

            ]
# print(df_private.columns)


# Creat new columns from the list and Fill the following columns with 'No'


fill_col = ['Charter','Magnet','Title I School','Title 1 School Wide']

for col in fill_col:
    df_private[col] = 'No'

# Drop the columns that are not needed
df_private = df_private.drop(columns=drop_col)




In [ ]:
## Make some changes to the public school data

# Rename columns to match private school data
# Rename Students to HS_ENROLL

df_public = df_public.rename(columns={'Students': 'HS_ENROLL'})

# Create and fill some columns that are from the private school data
df_public = df_public.assign(
    **{'Coed?': 'Coed',
    'Relig': 'None',
    'Community_Type': df_public['Locale'].str.split(':').str[0],
    'Relig_subtype': 'None'}
)



In [ ]:
# Now that the dataframes have some columns with the same names, we can concatenate them
df_combined = pd.concat([df_public, df_private], ignore_index=True)

In [ ]:
## Make a copy and output a csv of the dataframe for checking
df_combined_copy = df_combined.copy()
df_combined_copy.to_csv('TEMP/clean_tables/school_master_table.csv', index=False)

In [ ]:
matches_df.head()

## See the distrobution of scores

matches_df['score'].hist(bins=10)

In [9]:
import pandas as pd
from fuzzywuzzy import process

# Create a list to hold the matches
matches = []

# Loop through the game teams and find the best match in the school names list
for i, team in enumerate(game_teams):
    match = process.extractOne(team, school_names)
    # If match is None, append a placeholder value
    if match is None:
        matches.append((None, None, i))
    else:
        matches.append((*match, i))

# Create a dataframe from the matches list
matches_df = pd.DataFrame(matches, columns=['match', 'score', 'index'])

# Add the game team names to the dataframe
matches_df['game_team'] = game_teams

# Add the school names to the dataframe
matches_df['school_name'] = matches_df['match']

# # Transform the text columns to title case
# matches_df['game_team'] = matches_df['game_team'].str.title()
# matches_df['school_name'] = matches_df['school_name'].str.title()
# matches_df['match'] = matches_df['match'].str.title()


# Displays the first 5 rows of the dataframe
matches_df.head()

# Save as a csv
matches_df.to_csv('data\MHSAA\school_info\school_matches_TEST_NEW_222.csv', index=False)




In [ ]:
matches_df['score'].describe()

# Show the rows where the score is less than 80
print(f"Number of matches below  80: {len(matches_df[matches_df['score'] < 80])}")
print(f"Number of matches between 80 and 90: {len(matches_df[(matches_df['score'] >= 80) & (matches_df['score'] < 90)])}")
print(f"Number of matches above 90: {len(matches_df[(matches_df['score'] >= 90) & (matches_df['score'] < 100)])}")

print(f"Total number of matches: {len(matches_df)}")

In [ ]:
df_combined.columns

df_combined.head()

In [ ]:
# df_combined.sample(20)

# df_combined.shape

# df_combined.columns

In [ ]:
## Create a list of NCES IDS from a csv

csv_path ='TEMP/nces_public_data_full.csv'

nces_ids = pd.read_csv(csv_path)

nces_ids.head()

# save the NCES IDs as a list
nces_ids_list = nces_ids['NCES School ID'].tolist()

print(len(nces_ids_list))


## Code above working
## Blow is an attempt to get and store the demographic data for the public schools in the data set



In [ ]:
# # Use the NCES website to get the demographic and enrollment data for each public school

# ## Going to probably do this by building URLS to each school's page

# # NCES URL BUILDER

# base_url = 'https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&SchoolID='

# mid_url = '&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID='

# Test_ID = '262115005792'
# # School ID goes in between the two strings and after the second string

# nces_url_test = base_url + Test_ID + mid_url + Test_ID

# print(nces_url_test)



In [ ]:
# # try to read the url with pandas
# df_test = pd.read_html(nces_url_test)

# print(df_test)
df_public.columns

In [13]:
## Get a list of the school Ids for all the public schools
nces_ids_list = df_public['NCES School ID'].tolist()

df_public_ids[0:5]

[262016008622, 260033004509, 260858008255, 260016601035, 262313008413]

In [ ]:
# get the demographic catagories
# demo_cat = 
df_test[12].iloc[0,1:8]
# demo_counts = df_test[12].iloc[1,1:8]

In [14]:
### PARSE PUBLIC SCHOOL DEMO DATA FROM NCES WEBSITE

import pandas as pd
import requests
from bs4 import BeautifulSoup

def parse_school_data(school_ids):
    # Start with an empty DataFrame to store the results
    df = pd.DataFrame()

    for school_id in nces_ids_list:
        # Build the URL for this school
        url = f"https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&SchoolID={school_id}&ID={school_id}&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID2={school_id}"

        # Request the page and parse it with BeautifulSoup
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')

        # Get the value in the first column of the first row and drop everything in from of the \t
        school_name = df_test[2].iloc[0,0].split('\t')[1]
        school_id = df_test[2].iloc[0,2].split('ID:')[1].strip()

        school_district = df_test[2].iloc[2,0].split('\t')[1].split('district information')[0]
        school_district_id = df_test[2].iloc[2,2].split('ID:')[1].strip()

        #Get Total Students
        total_students = df_test[6].iloc[0,2]
        # Get Teachers at the school
        teachers = df_test[6].iloc[1,2]
        # Get the student/teacher ratio
        student_teacher_ratio = df_test[6].iloc[2,2]


        # Get all the grade levels and counts
        grade_levels = df_test[10].iloc[0,1:7]
        grade_counts = df_test[10].iloc[1,1:7]
        grade_dict = dict(zip(grade_levels, grade_counts))

        # Get the demographic categories and counts
        demo_cat = df_test[12].iloc[0,1:8]
        demo_counts = df_test[12].iloc[1,1:8]
        demo_dict = dict(zip(demo_cat, demo_counts))

        # Get the gender categories and counts
        gender_cat = df_test[14].iloc[0,1:3]
        gender_counts = df_test[14].iloc[1,1:3]
        gender_dict = dict(zip(gender_cat, gender_counts))

        # Save the parsed data to a row in a dataframe
        df_test_row = pd.DataFrame({'school_name': school_name,
        'school_id': school_id,
        'school_district': school_district,
        'school_district_id': school_district_id,
        'total_students': total_students,
        'teachers': teachers,
        'student_teacher_ratio': student_teacher_ratio,
        'lunch_free_count': lunch_free_count,
        'lunch_reduced_count': lunch_reduced_count,
        'lunch_total_count': lunch_total_count,
        **grade_dict,
        **demo_dict,
        **gender_dict
        }, index=[0])

        # Append the new row to our DataFrame
        df = pd.concat([df, df_test_row], ignore_index=True)

    return df



In [15]:
### Feed the list of school ids into the function



df = parse_school_data(nces_ids_list)


NameError: name 'df_test' is not defined

In [ ]:
df.head(10)

In [ ]:
## save the results to a csv file
df.to_csv('school_data_OVERNIGHT_PARSE_MANUAL CHECK NEEDED.csv', index=False)

In [ ]:
# # # Table 2 has the school identify infor (name, address, NCES code, etc.)
# df_test[2].head()
# df_test[2].head()

# Get the value in the first column of the first row and drop everything in from of the \t
school_name = df_test[2].iloc[0,0].split('\t')[1]
school_id = df_test[2].iloc[0,2].split('ID:')[1].strip()

school_district = df_test[2].iloc[2,0].split('\t')[1].split('district information')[0]
school_district_id = df_test[2].iloc[2,2].split('ID:')[1].strip()


# # Table 5 - county, locale,
# df_test[5].head()
# print(df_test[5].head())

# Table 6  total students, student/teacher ratio
df_test[6].head()
print(df_test[6].head())
#Get Total Students
total_students = df_test[6].iloc[0,2]
# Get Teachers at the school
teachers = df_test[6].iloc[1,2]
# Get the student/teacher ratio
student_teacher_ratio = df_test[6].iloc[2,2]





## Table 10 is the enrollment by grade table nice an clean
# df_test[10].head()
# print(df_test[10].head())

# Get all the grade levels and counts
grade_levels = df_test[10].iloc[0,1:7]
grade_counts = df_test[10].iloc[1,1:7]

# # ## Table 12 is enrollment by demogrtaohic (raw count) nice and clean
# df_test[12].head()
# print(df_test[12].head())

# get the demographic catagories
demo_cat = df_test[12].iloc[0,1:8]
demo_counts = df_test[12].iloc[1,1:8]

# Table 14 is enrollment by gender (raw count) nice and clean
# df_test[14].head()
# print(df_test[14].head())

# Get catagory names
gender_cat = df_test[14].iloc[0,1:3]
gender_counts = df_test[14].iloc[1,1:3]

# # Table 16 is free and reduced lunch eligibility (raw count) - not clean. will need further processing
# df_test[16].head()
# print(df_test[16].head())

# Get free and reduced lunch counts
lunch_free_count = df_test[16].iloc[0,0].split(':')[1].strip()
lunch_reduced_count = df_test[16].iloc[0,1].split(':')[1].strip()
lunch_total_count = df_test[16].iloc[0,2].split(':')[1].strip()

## Save the data to a row that can be appended to a dataframe


# Create a list of the data








# print(df_test[6].head())

In [ ]:
## Drop any nan columns
df_test[0].dropna(axis=1, how='all')
